# E-commerce Car shop application

The Json file is not validated as is .Its found that the array should have been seperated by " , "and start with square brackets but when each line is tested it is validated .Hence we are reading the lines one by one. One could also make the file validated by adding {} and "," after every entry . Next step is we parse the json txt in file1 and run loop to read every single line and that could be accessed in rowData

 - Input File:.Json file
 - rowData:Each entry of the .Json text of type <class 'dict'>
 - Module required:json

# Task 1:Data preprocessing

 - Objective - 
 Transform the supplier data to achieve the same granularity as the target data.
 - The json file is simply written in excel with no changes made
 - Modules: xlsxwriter,xlrd,xlwt,xlutils and xl_copy

In [1]:
import json
import xlsxwriter
import xlrd, xlwt
from xlutils.copy import copy as xl_copy

## Used for Normalizing color values
ColorMapping = {"gold":"gold","grn":"green", "schwarz":"black", "rot":"red", "silber":"silver", "anthrazit":"anthracite","weiss":"white","blau":"blue","grau":"grey","gelb":"yellow","violett":"voilet","grn":"green","orange":"orange","beige":"beige","bordeaux":"bordeaux","braun":"brown"}

## Used for Normalizing DriverType values
DriverTypeMapping= {"Allrad":"1","Hinterradantrieb":"2","Vorderradantrieb":"3","null":"0"}

## Target schema and its default value
TargetSchemaMap = {"carType":"NewSupplier","color":"Unknown","condition":"NewSupplier","currency":"NewSupplier","drive":"NewSupplier","city":"Unknown","country":"NewSupplier","make":"NewSupplier","manufacture_year":"NewSupplier","mileage":"NewSupplier","mileage_unit":"NewSupplier","model":"NewSupplier","model_variant":"NewSupplier","price_on_request":"NewSupplier","type":"NewSupplier","zip":"NewSupplier","manufacture_month":"NewSupplier","fuel_consumption_unit":"NewSupplier"}

## Utilify functions
def stripNonAlphaNum(text):
    import re
    list = re.compile(r'\W+').split(text)
    stripped_string = re.sub(r'\W+', '', list[0])
    alphanumeric = ""

    for character in stripped_string:
        if ord(character) >= 65 and ord(character) <= 90:
             alphanumeric += character
            ## checking for lower case
        elif ord(character) >= 97 and ord(character) <= 122:
             alphanumeric += character
    return alphanumeric.lower()

def mynormalize(dataType, text):
    if dataType=="color":
        germanColor = stripNonAlphaNum(text)
        return ColorMapping[germanColor]
    elif dataType=="drive":
        return  DriverTypeMapping[text]
    elif dataType=="milage":
        return  text.split(" ")[0]

## Final output in MasterExcel.xlsx
workbook = xlsxwriter.Workbook('MasterExcel.xlsx')

### Add 1st Tab/Sheet now
worksheet = workbook.add_worksheet()
row = 0
col = 0
row+=1
file1 = open('supplier_car.json', 'r')
Lines = file1.readlines()
headingAdded = False
for line in Lines:
    r = line.strip()
    rowData = json.loads(r)
    if headingAdded == False:
        for keyInSuppliersData in rowData:
            worksheet.write(row, col,keyInSuppliersData)
            col+=1
            headingAdded = True
        row+=1
    col=0
    for keyInSuppliersData in rowData:
        #print("key is {} and its valye is {}".format(keyInSuppliersData, rowData[keyInSuppliersData]))
        worksheet.write(row, col,rowData[keyInSuppliersData])
        col+=1
    row+=1

### Add 2nd Tab/Sheet now

worksheet2 = workbook.add_worksheet()
row = 0
col = 0
row+=1
headingAdded = False
for line in Lines:
    r = line.strip()
    rowData = json.loads(r)
    if headingAdded == False:
        for keyInSuppliersData in rowData:
            worksheet2.write(row, col,keyInSuppliersData)
            col+=1
            headingAdded = True
        row+=1
    col=0
    for keyInSuppliersData in rowData:
        if keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "BodyColorText":
            #print("in normalize")
            worksheet2.write(row, col,rowData[keyInSuppliersData])
            col+=1
            suppliedBodyColr =  mynormalize("color", rowData["Attribute Values"])
            #print("val is {}".format(suppliedBodyColr))
            worksheet2.write(row, col,suppliedBodyColr)
            col+=1
        elif keyInSuppliersData == "Attribute Values" and rowData["Attribute Names"] == "BodyColorText":
            #print("skipped")
            continue
        elif keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "DriveTypeText":
            worksheet2.write(row, col,rowData[keyInSuppliersData])
            col+=1
            normalizedDriverType = mynormalize("drive", rowData["Attribute Values"])
            worksheet2.write(row, col, normalizedDriverType)
            col+=1
        elif keyInSuppliersData == "Attribute Values" and rowData["Attribute Names"] == "DriveTypeText":
            #print("skipped")
            continue
        elif keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "ConsumptionTotalText":
            worksheet2.write(row, col,rowData[keyInSuppliersData])
            col+=1
            normalizedMilage = mynormalize("milage", rowData["Attribute Values"])
            worksheet2.write(row, col, normalizedMilage)
            col+=1
        elif keyInSuppliersData == "Attribute Values" and rowData["Attribute Names"] == "ConsumptionTotalText":
            #print("skipped")
            continue
        else:
            worksheet2.write(row, col,rowData[keyInSuppliersData])
            col+=1
    row+=1

### Add 3rd Tab/Sheet now

worksheet3 = workbook.add_worksheet()
row = 0
col = 0

# Add the Target schema header into excel
for keyInTargetschema in TargetSchemaMap:
    worksheet3.write(row, col, keyInTargetschema)
    col+=1

row+=1
col=0
for line in Lines:
    r = line.strip()
    rowData = json.loads(r)
    
    # populate local object with only required values (like color, city, make, model and model_variant) from rowData (i.e. supplier data)
    localRetrievedData = {}
    for keyInSuppliersData in rowData:
        #print("key is {} and its value is {}".format(keyInSuppliersData, rowData[keyInSuppliersData]))        
        if keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "BodyColorText":
            localRetrievedData["color"]=mynormalize("color", rowData["Attribute Values"])
        if keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "ConsumptionTotalText":
            localRetrievedData["mileage"]=mynormalize("milage", rowData["Attribute Values"])
        elif keyInSuppliersData == "Attribute Names" and rowData[keyInSuppliersData] == "City":
            localRetrievedData["city"]=rowData["Attribute Values"]
        elif keyInSuppliersData == "MakeText":
            localRetrievedData["make"]=rowData[keyInSuppliersData]
        elif keyInSuppliersData == "ModelText":
            localRetrievedData["model"]=rowData[keyInSuppliersData]
        elif keyInSuppliersData == "ModelTypeText":
            localRetrievedData["model_variant"]=rowData[keyInSuppliersData]

    # Now fill the worsheet with values localRetrievedData, if exists. Else add default values defined in TargetSchemaMap.
    col=0
    for keyInTargetschema in TargetSchemaMap:
        if keyInTargetschema in localRetrievedData:
            worksheet3.write(row, col,localRetrievedData[keyInTargetschema])
        else:
            worksheet3.write(row, col,TargetSchemaMap[keyInTargetschema])     
        col+=1

    row+=1

## close the workbook. This is the end.
workbook.close()



- Output:The .json is parsed and each lines are read and entries are put in the Excel sheet "MasterExcel.xlsx
 - This MasterExcel can then be used to have a sneakpeak of every details of matrix and one can apply various filtering techniques.
 - The .xlsx file is read using pandas to give a snapshot of the view of supplier data.

In [2]:
import pandas as pd
xls = pd.ExcelFile('MasterExcel.xlsx')
df1 = pd.read_excel(xls, 'Sheet1')
df1=pd.read_excel("MasterExcel.xlsx")
new_header = df1.iloc[0] #grab the first row for the header
df1 = df1[1:] #take the data less the header row
df1.columns = new_header #set the header row as the df header
df1.head()

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
1,976.0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
2,1059.0,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
3,524.0,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
4,608.0,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
5,726.0,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mét.,002d30c2-43f6-4905-868f-160dbc445c56


# Task 2: Data Normalisation

 - Normalisation :From the above data it is evident that Normalisation is required in order to achieve the similar meaning with the target dataset.
     - I have chosen "DriverType" and the "BodyColorText" from the "AttributeNames" and Normalised.
          - I have set the values in the "DriverType" "Allrad" as 1","Hinterradantrieb" as "2","Vorderradantrieb" as "3" and "null":"0".
          - Similarly the BodyColorText had German words and I have transformed to English colors just to give the advantage of pulling in the International customers.



 - These two Attribute names are normalized and mapped accordingly and put into sheet 2 of the MasterExcel.
 - There is potential to Normalize many other Attributes such as ConsumptionRateText which has attribute value say 14.4 1/100km to just the numbers which could be used in Data analysis to estimate the which models are best in termns of fuel consumption.
 - Also One could normalize the TypeNameFull to decrease the complexity in retaining the full name.

Sheet 2 has Normalised Supplier data reflecting normalization of two attributes.

In [3]:
df2 = pd.read_excel(xls, 'Sheet2')
new_header = df2.iloc[0] #grab the first row for the header
df2 = df2[1:] #take the data less the header row
df2.columns = new_header #set the header row as the df header
df2.head()

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
1,976.0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
2,1059.0,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
3,524.0,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
4,608.0,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
5,726.0,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,black,002d30c2-43f6-4905-868f-160dbc445c56


# Task 3: Data Integration

 - Input: normalised supplier data
 - Output: integrated supplier data
      - The supplier data with a specific data schema is transformed into a new dataset with target data schema

Sheet 3 has target schema data which has the potential to fill in the desired attribute or records depending on the demands.

 - The new dataset is the target schema which essentially has all the original target datasets and in addition to it 4 more records from the supplier dataset is mapped into the target.
 - When the information is required one could possibly get the information from the suppliers and fill in the essentials.


In [7]:
df4=pd.read_excel(xls,"Sheet3")
df4.head()

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,NewSupplier,Unknown,NewSupplier,NewSupplier,NewSupplier,Unknown,NewSupplier,MERCEDES-BENZ,NewSupplier,NewSupplier,NewSupplier,SLR,SLR McLaren,NewSupplier,NewSupplier,NewSupplier,NewSupplier,NewSupplier
1,NewSupplier,Unknown,NewSupplier,NewSupplier,NewSupplier,Unknown,NewSupplier,MERCEDES-BENZ,NewSupplier,NewSupplier,NewSupplier,ML 350,ML 350 Inspiration,NewSupplier,NewSupplier,NewSupplier,NewSupplier,NewSupplier
2,NewSupplier,Unknown,NewSupplier,NewSupplier,NewSupplier,Unknown,NewSupplier,AUDI,NewSupplier,NewSupplier,NewSupplier,S6,S6 Avant quattro 4.2,NewSupplier,NewSupplier,NewSupplier,NewSupplier,NewSupplier
3,NewSupplier,Unknown,NewSupplier,NewSupplier,NewSupplier,Unknown,NewSupplier,SAAB,NewSupplier,NewSupplier,NewSupplier,9-3,9-3 2.0i-16 TS Aero,NewSupplier,NewSupplier,NewSupplier,NewSupplier,NewSupplier
4,NewSupplier,black,NewSupplier,NewSupplier,NewSupplier,Unknown,NewSupplier,PORSCHE,NewSupplier,NewSupplier,NewSupplier,911,911 Turbo Cabrio,NewSupplier,NewSupplier,NewSupplier,NewSupplier,NewSupplier


# Deliverables:

 - With this code An Excelsheet workbook with 3 worksheets showing the results of each
step above (i.e., pre-pro- cessing/normalisation/integration) are saved in Sheet 1,Sheet2 and Sheet3 resüectively
 - It can also be converted to pandas and different Data Analysis can be done in future.